In [ ]:
missing_entries = 0

def generate_label(price_movement) -> int:
    global missing_entries
    if price_movement <= -0.005:
        return 0
    elif price_movement > 0.0055:
        return 1
    else:
        missing_entries += 1
        return -1

import pandas as pd

df = pd.read_csv('/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/price/preprocessed/csv/ABB.csv')
path = '/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/price/preprocessed/csv/ABB.csv'

stock_name = path.split('/')[-1].split('.')[0]

df['Name'] = stock_name
df['Label'] = df.apply(lambda row: generate_label(row['Movement_Percent']), axis=1)
print(len(df))

df = df[~(df['Label'] == -1)]

print(len(df))

df['Date'] = pd.to_datetime(df['Date'])

# Define start and end dates
start_date = '2014-01-01'
end_date = '2015-08-01'

# Filter the DataFrame for dates between start_date and end_date
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

print(filtered_df)

In [ ]:
import pandas as pd

def get_train_data(path:str) -> pd.DataFrame:
    df = pd.read_csv(path)

    stock_name = path.split('.')[0]
    if '/' in stock_name:
        stock_name = stock_name.split('/')[-1]

    df['Name'] = stock_name

    df['Label'] = df.apply(lambda row: generate_label(row['Movement_Percent']), axis=1)

    df = df[~(df['Label'] == -1)]

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date')

    # Define start and end dates
    start_date = '2014-01-01'
    end_date = '2015-08-01'
    #start_date = '2015-08-01'
    #start_date = '2015-10-01'
    #end_date = '2016-01-01'
    # Filter the DataFrame for dates between start_date and end_date
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    filtered_df = filtered_df.drop_duplicates()
    return filtered_df



In [ ]:
import os
missing_entries = 0
root = '/home/skage/projects/ikt450_deep-neural-networks/stocknet-project-ikt450/dataset/price/preprocessed/csv/'
for idx, stock_path in enumerate(os.listdir(root)):
    df_stock = get_train_data(os.path.join(root, stock_path))
    if idx == 0:
        df_train = df_stock
    else:
        df_train = pd.concat([df_train, df_stock])
    
print(len(df_train))
print(missing_entries)
print(df_train.head())
    

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['Name', 'Date']).reset_index(drop=True)

lag = 5

lagged_columns = []
def create_lagged_features(group):
    for i in range(lag, 0, -1):
        group[f'Open_t_minus_{i}'] = group['Open'].shift(i)
        group[f'Close_t_minus_{i}'] = group['Close'].shift(i)
        # Append only if columns are new to avoid duplicates
        if f'Open_t_minus_{i}' not in lagged_columns:
            lagged_columns.append(f'Open_t_minus_{i}')
        if f'Close_t_minus_{i}' not in lagged_columns:
            lagged_columns.append(f'Close_t_minus_{i}')
    return group

# Apply the function to each group
df = df.groupby('Name').apply(create_lagged_features)

# Drop rows with NaN values in any of the lagged columns
df = df.dropna(subset=lagged_columns).reset_index(drop=True)

print(df.head())

In [ ]:
df = df_train

lag = 5



In [ ]:
from sklearn.preprocessing import LabelEncoder

def create_rnn_input(df):
    # Ensure 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])

    # Sort the DataFrame by 'Name' and 'Date'
    df = df.sort_values(['Name', 'Date']).reset_index(drop=True)

    # List to store the new rows
    data_rows = []

    name_encoder = LabelEncoder()
    df['Name'] = name_encoder.fit_transform(df['Name'])

    # Group the DataFrame by 'Name' (stock)
    grouped = df.groupby('Name')

    # For each stock group
    for name, group in grouped:
        group = group.sort_values('Date').reset_index(drop=True)

        # Skip if the group has less than 6 rows (need at least 5 days of history)
        if len(group) < 6:
            continue

        # Iterate over the group starting from the 5th index
        for idx in range(5, len(group)):
            # Get the previous 5 trading days
            prev_data = group.iloc[idx-5:idx]

            # Prepare the row data
            row_dict = {'Date': group.loc[idx, 'Date'], 'Name': name}

            # Add Open and Close prices from previous 5 trading days
            for i in range(5):
                row_dict[f'Open_t-{5-i}'] = prev_data.iloc[i]['Open']
                row_dict[f'Close_t-{5-i}'] = prev_data.iloc[i]['Close']

            # Add the Label for the target date
            row_dict['Label'] = group.loc[idx, 'Label']

            # Append the row to data_rows
            data_rows.append(row_dict)

    # Create a new DataFrame from data_rows
    new_df = pd.DataFrame(data_rows)

    x = new_df.drop(['Label', 'Date'], axis=1)
    y = new_df['Label']
    x = x.to_numpy()
    y = y.to_numpy()

    return x, y

new_df = create_rnn_input(df)
print(new_df.head())
print(len(df), len(new_df))